# Time Dependent Route Comparison: shadow routes

In [1]:
import geopandas as gpd
from shapely.geometry import LineString, Point
import numpy as np
from matplotlib import pyplot as plt
from importlib import reload
import contextily as ctx
from pathlib import Path
import pandas as pd
import datetime as dt
from ipyleaflet import Map, GeoData, basemaps, LayersControl,Icon, LegendControl,ScaleControl

In [2]:
import ors_evaluation

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
ors_url = "https://meingruen.openrouteservice.org/ors/" 

### Routing parameters

In [5]:
profile = "foot-walking"

In [6]:
start_coords = (51.04387630433032, 13.76028997302245)
destination_coords = (51.04619669418911, 13.74123538289695)

In [7]:
body_normal = {'coordinates': [start_coords[::-1], destination_coords[::-1]],
          'instructions': 'false',
          'preference': 'recommended',
          "extra_info": ["green"],
          "elevation": "true",
          "continue_straight": "true",
          "options": {"avoid_features": ["ferries"], 
                      "profile_params": {"weightings": {
                          "steepness_difficulty":1, 
                          "shadow":0}}}
        
    }
body_shadow = {'coordinates': [start_coords[::-1], destination_coords[::-1]],
          'instructions': 'false',
          'preference': 'recommended',
          "extra_info": ["green"],
          "elevation": "true",
          "continue_straight": "true",
          "options": {"avoid_features": ["ferries"], 
                      "profile_params": {"weightings": {
                          "steepness_difficulty":1, 
                          "shadow":1}}}
    }

### Calculate initial routes

In [8]:
route_normal = ors_evaluation.Route(params=body_normal, base_url=ors_url, profile=profile, fmt="geojson")
route_shadow = ors_evaluation.Route(params=body_shadow, base_url=ors_url, profile=profile, fmt="geojson")

In [9]:
def handle_start_change(**kwargs):
    global destination_coords
    global start_coords
    global m
    start_coords = kwargs["location"]
    
    body_normal["coordinates"] = [start_coords[::-1], destination_coords[::-1]]
    body_shadow["coordinates"] = [start_coords[::-1], destination_coords[::-1]]

    route_normal = ors_evaluation.Route(params=body_normal, base_url=ors_url, profile=profile, fmt="geojson")
    route_shadow = ors_evaluation.Route(params=body_shadow, base_url=ors_url, profile=profile, fmt="geojson")
    
    m.remove_layer(m.layers[-1])
    m.remove_layer(m.layers[-1])

    route_layer_normal = GeoData(geo_dataframe = route_normal.as_dataframe(),
                            name = 'Route Normal', 
                            style={"color": "red",
                                   "opacity": 0.5})
    route_layer_shadow = GeoData(geo_dataframe = route_shadow.as_dataframe(),
                            name = 'Route Shadow',
                           style={"color": "blue",
                                   "opacity": 0.5})
    m.add_layer(route_layer_normal)
    m.add_layer(route_layer_shadow)

In [10]:
def handle_destination_change(**kwargs):
    #route = ors_evaluation.Route(params=body, base_url=ors_normal, profile=profile, fmt=fmt)
    global destination_coords
    global start_coords
    global m
    destination_coords = kwargs["location"]
    
    body_normal["coordinates"] = [start_coords[::-1], destination_coords[::-1]]
    body_shadow["coordinates"] = [start_coords[::-1], destination_coords[::-1]]

    route_normal = ors_evaluation.Route(params=body_normal, base_url=ors_url, profile=profile, fmt="geojson")
    route_shadow = ors_evaluation.Route(params=body_shadow, base_url=ors_url, profile=profile, fmt="geojson")
    
    m.remove_layer(m.layers[-1])
    m.remove_layer(m.layers[-1])

    route_layer_normal = GeoData(geo_dataframe = route_normal.as_dataframe(),
                            name = 'Route Normal', 
                            style={"color": "red",
                                   "opacity": 0.5})
    route_layer_shadow = GeoData(geo_dataframe = route_shadow.as_dataframe(),
                            name = 'Route Shadow',
                           style={"color": "blue",
                                   "opacity": 0.5})
    m.add_layer(route_layer_normal)
    m.add_layer(route_layer_shadow)

### Map showing routes

In [11]:
#icon_a = Icon(icon_url='./icons/icons8-marker-a-96.png',icon_size=[25, 30], icon_anchor=[10,25])
#icon_b = Icon(icon_url='./icons/icons8-marker-b-96.png',icon_size=[25, 30], icon_anchor=[10,25])

In [12]:
from ipyleaflet import Map, Marker

center = (min(start_coords[0], destination_coords[0]) + abs(start_coords[0] - destination_coords[0]), 
          min(start_coords[1], destination_coords[1]) + abs(start_coords[1] - destination_coords[1]))

m = Map(center=center, zoom=14, scroll_wheel_zoom=True, basemap=basemaps.CartoDB.Positron)

start = Marker(location=start_coords, draggable=True)
m.add_layer(start);

destination = Marker(location=destination_coords, draggable=True)
m.add_layer(destination);

route_layer_normal = GeoData(geo_dataframe = route_normal.as_dataframe(),
                        name = 'Route Normal', 
                        style={"color": "red",
                               "opacity": 0.5})
route_layer_shadow = GeoData(geo_dataframe = route_shadow.as_dataframe(),
                        name = 'Route Shadow',
                       style={"color": "blue",
                               "opacity": 0.5})

m.add_layer(route_layer_normal)
m.add_layer(route_layer_shadow)
    
start.on_move(handle_start_change)
destination.on_move(handle_destination_change)

legend = LegendControl({"Normal route": "red", "Shadow route": "blue"}, name="Legend", position="topright")
m.add_control(legend)

m.add_control(ScaleControl(position='bottomleft'))

m

Map(center=[51.04619669418911, 13.76028997302245], controls=(ZoomControl(options=['position', 'zoom_in_text', …